In [ ]:
# this code is used for GPU
# train_set is in R^{m_train x d} and test_set is in R^{m_test x d} , m_train is the size of traing set, m_test is the size of test set, d is dimension of features.




# data setup
X_train=train_set
y_train=train_label
X_test=test_set
y_test=test_label
num_classes = 


import numpy as np
import cupy as cp

layers=10
eta=0.5
eps=0.1


def forward(F, Z, y):
    gam=compute_gam(y)
    m,d=Z.shape
    #F=np.eye(d) #初始化矩阵为单位矩阵
    c=d / (m * eps)
    preE=cp.linalg.inv(cp.eye(m)+c*Z@F.T@F@Z.T)
    E=c* (cp.eye(d)-c*F@Z.T@preE@Z@F.T)
    expd=E @ F @ Z.T @ Z
    for j in range(num_classes):
        Z_j=Z[y==j]
        m_j=Z_j.shape[0]
        c_j=d / (m_j * eps)
        preC=cp.linalg.inv(cp.eye(m_j)+c_j*Z_j@F.T@F@Z_j.T)
        Cj=c_j*(cp.eye(d)-c_j*F@Z_j.T@preC@Z_j@F.T)
        if j==0:
            clus=gam[j]* Cj @ F @ Z_j.T @ Z_j
        else:
            clus=clus+ gam[j]* Cj @ F @ Z_j.T @ Z_j
    F=F+eta*(expd-clus)
    return F,Z,y

def compute_gam( y):
    m_j = [(y==j).nonzero()[0].size for j in range(num_classes)]
    gam = np.array(m_j) / y.size
    return gam

def compute_loss(F, Z, y):
    gam=compute_gam(y)
    m, d = Z.shape
    I = np.eye(d)
    c = d / (m * eps)
    logdet = cp.linalg.slogdet(I + c * F@Z.T @ Z @F.T)[1]
    loss_expd = logdet / 2.

    loss_comp = 0.
    for j in np.arange(num_classes):
        idx = (y == int(j))
        Z_j = Z[idx, :]
        m_j = Z_j.shape[0]
        if m_j == 0:
            continue
        c_j = d / (m_j * eps)
        logdet_j = cp.linalg.slogdet(I + c_j *F@ Z_j.T @ Z_j@F.T)[1]
        loss_comp += gam[j] * logdet_j / 2.
    return loss_expd - loss_comp, loss_expd, loss_comp

def normalize(X, p=2):
    axes = tuple(np.arange(1, len(X.shape)).tolist())
    norm = np.linalg.norm(X.reshape(X.shape[0], -1), axis=1, ord=p)
    norm = np.clip(norm, 1e-8, np.inf)
    return X / np.expand_dims(norm, axes)

#layers=10
import argparse
import os
import time

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt





F=cp.eye(X_train.shape[1])

Z=X_train
y=y_train
start_time=time.time()
for i in range(layers):
    F,Z,y=forward(F,Z,y)
    #loss_total,loss_expd,loss_comp=compute_loss(F,Z,y)
    #print(f"layer: {i} | loss_total: {loss_total:5f} | loss_expd: {loss_expd:5f} | loss_comp: {loss_comp:5f}")
end_time=time.time()

Z_train=X_train@F.T
Z_test=X_test@F.T


Z_train=cp.asnumpy(Z_train)
y_train=cp.asnumpy(y_train)
Z_test=cp.asnumpy(Z_test)
y_test=cp.asnumpy(y_test)

Z_train=normalize(Z_train)
Z_test=normalize(Z_test)


##evaluate
import argparse
import os
import torch
import numpy as np

from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

def svm(train_features, train_labels, test_features, test_labels):
    svm = LinearSVC(verbose=0, random_state=10)
    svm.fit(train_features, train_labels)
    acc_train = svm.score(train_features, train_labels)
    acc_test = svm.score(test_features, test_labels)
    print("SVM: {}".format(acc_test))
    return acc_train, acc_test

def knn(train_features, train_labels, test_features, test_labels, k=5):
    """Perform k-Nearest Neighbor classification using cosine similaristy as metric.
    Options:
        k (int): top k features for kNN

    """
    sim_mat = train_features @ test_features.T
    topk = torch.from_numpy(sim_mat).topk(k=k, dim=0)
    topk_pred = train_labels[topk.indices]
    test_pred = torch.tensor(topk_pred).mode(0).values.detach()

    #print("knn",test_pred[0])

    acc = compute_accuracy(test_pred.numpy(), test_labels)
    print("kNN: {}".format(acc))
    return acc

def nearsub(train_features, train_labels, test_features, test_labels, n_comp=10):
    """Perform nearest subspace classification.

    Options:
        n_comp (int): number of components for PCA or SVD

    """
    scores_svd = []
    classes = np.unique(test_labels)
    features_sort, _ = sort_dataset(train_features, train_labels,
                                          classes=classes, stack=False)
    fd = features_sort[0].shape[1]
    if n_comp >= fd:
        n_comp = fd - 1
    for j in np.arange(len(classes)):
        svd = TruncatedSVD(n_components=n_comp).fit(features_sort[j])
        svd_subspace = svd.components_.T
        svd_j = (np.eye(fd) - svd_subspace @ svd_subspace.T) \
                        @ (test_features).T
        score_svd_j = np.linalg.norm(svd_j, ord=2, axis=0)
        scores_svd.append(score_svd_j)
    test_predict_svd = np.argmin(scores_svd, axis=0)
    ###
    #print('predict_svd',test_predict_svd[0])
    ###
    acc_svd = compute_accuracy(classes[test_predict_svd], test_labels)
    print('SVD: {}'.format(acc_svd))
    return acc_svd

def nearsub_pca(train_features, train_labels, test_features, test_labels, n_comp=10):
    """Perform nearest subspace classification.

    Options:
        n_comp (int): number of components for PCA or SVD

    """
    scores_pca = []
    classes = np.unique(test_labels)
    features_sort, _ = sort_dataset(train_features, train_labels,
                                          classes=classes, stack=False)
    fd = features_sort[0].shape[1]
    if n_comp >= fd:
        n_comp = fd - 1
    for j in np.arange(len(classes)):
        pca = PCA(n_components=n_comp).fit(features_sort[j])
        pca_subspace = pca.components_.T
        mean = np.mean(features_sort[j], axis=0)
        pca_j = (np.eye(fd) - pca_subspace @ pca_subspace.T) \
                        @ (test_features - mean).T
        score_pca_j = np.linalg.norm(pca_j, ord=2, axis=0)
        scores_pca.append(score_pca_j)
    test_predict_pca = np.argmin(scores_pca, axis=0)
    acc_pca = compute_accuracy(classes[test_predict_pca], test_labels)
    print('PCA: {}'.format(acc_pca))
    return acc_svd

def compute_accuracy(y_pred, y_true):
    """Compute accuracy by counting correct classification. """
    assert y_pred.shape == y_true.shape
    return 1 - np.count_nonzero(y_pred - y_true) / y_true.size

def baseline(train_features, train_labels, test_features, test_labels):
    test_models = {'log_l2': SGDClassifier(loss='log', max_iter=10000, random_state=42),
                   'SVM_linear': LinearSVC(max_iter=10000, random_state=42),
                   'SVM_RBF': SVC(kernel='rbf', random_state=42),
                   'DecisionTree': DecisionTreeClassifier(),
                   'RandomForrest': RandomForestClassifier()}
    for model_name in test_models:
        test_model = test_models[model_name]
        test_model.fit(train_features, train_labels)
        score = test_model.score(test_features, test_labels)
        print(f"{model_name}: {score}")

def sort_dataset(data, labels, classes, stack=False):
    """Sort dataset based on classes.

    Parameters:
        data (np.ndarray): data array
        labels (np.ndarray): one dimensional array of class labels
        classes (int): number of classes
        stack (bol): combine sorted data into one numpy array

    Return:
        sorted data (np.ndarray), sorted_labels (np.ndarray)

    """
    if type(classes) == int:
        classes = np.arange(classes)
    sorted_data = []
    sorted_labels = []
    for c in classes:
        idx = (labels == c)
        data_c = data[idx]
        labels_c = labels[idx]
        sorted_data.append(data_c)
        sorted_labels.append(labels_c)
    if stack:
        sorted_data = np.vstack(sorted_data)
        sorted_labels = np.hstack(sorted_labels)
    return sorted_data, sorted_labels




_, acc_svm = svm(Z_train, y_train, Z_test, y_test)
acc_knn = knn(Z_train, y_train, Z_test, y_test, k=5)
acc_svd = nearsub(Z_train, y_train, Z_test, y_test, n_comp=5)
acc = {"svm": acc_svm, "knn": acc_knn, "nearsub-svd": acc_svd}

elapsed_time = end_time - start_time
print(f"Model execution time: {elapsed_time:.4f} seconds")